# Landing AI + Ravin = :)

The above message is pretty clear to me, but let's see if we can train a Neural Net to find that message in random noise

## Dataset
The input to our neural net will be a set of images that either contain our  important message or a string of random characters. For extra difficulty the random AI will have a higher occurence of :) emojis, + and = characters as well. You will need to install the LandingAI package if following along.

### Generate dataset
Let's go ahead and generate some images. In the Landing AI package there the image class contains all the code used to generate test and train images


In [13]:
%load_ext autoreload
%autoreload 2
from landingai import image
image.gen_images(train_examples=5000, test_examples=200, delete=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
train exists, deleting directory and images and creating new ones
test exists, deleting directory and images and creating new ones


### Data Load
Each image now needs to be loaded and labeled, and the target vector needs to be one hot encoded

In [14]:
%load_ext autoreload
%autoreload 2
from landingai import model_utils
x_train, y_train = model_utils.load_data("train")
x_test, y_test = model_utils.load_data("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### One Hot Encode Targets

In [15]:
from keras.utils import to_categorical

In [16]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Train Model

In [20]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, LeakyReLU
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(filters = 8, kernel_size=2, strides=2, padding='valid', input_shape=(270,270,1)))
model.add(LeakyReLU(alpha =.1))
model.add(MaxPooling2D(pool_size=4))

model.add(Conv2D(filters = 4, kernel_size=2, strides=2, padding='valid'))
model.add(LeakyReLU(alpha =.1))

model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 20, verbose=1)


Train on 5000 samples, validate on 200 samples
Epoch 1/20
5000/5000 [==============================] - 18s - loss: 0.3787 - acc: 0.8822 - val_loss: 0.0885 - val_acc: 0.9950
Epoch 2/20
5000/5000 [==============================] - 18s - loss: 0.0394 - acc: 0.9934 - val_loss: 0.0102 - val_acc: 1.0000
Epoch 3/20
5000/5000 [==============================] - 20s - loss: 0.0204 - acc: 0.9928 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 4/20
5000/5000 [==============================] - 20s - loss: 0.0115 - acc: 0.9960 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 5/20
5000/5000 [==============================] - 20s - loss: 0.0108 - acc: 0.9956 - val_loss: 0.0249 - val_acc: 0.9950
Epoch 6/20
5000/5000 [==============================] - 21s - loss: 0.0066 - acc: 0.9976 - val_loss: 0.0792 - val_acc: 0.9650
Epoch 7/20
5000/5000 [==============================] - 25s - loss: 0.0080 - acc: 0.9964 - val_loss: 5.7241e-04 - val_acc: 1.0000
Epoch 8/20
5000/5000 [==============================] - 27s - loss:

In [24]:
model.save("Classifier.hdf5")